In [1]:
import json
import requests
from datetime import datetime, date
import pandas as pd          # to read data in python with dataframe format
import sqlalchemy            # Toolkit and Object Relational Mapper that gives full power and flexibility of SQL
import matplotlib.pyplot as plt     # Visualization
import psycopg2
import psycopg2.extras
from sqlalchemy import create_engine

# Calling URL link request

In [2]:
# App Credentials

apps_id = 'xxxxxxxxx'
apps_key = 'xxxxxxxx'
source = 'JKT'
destination = 'SIN'
dod = '20190507'

# requests url link format
# {0} -> app_id, {1} -> app_key, {2} -> source, {3} -> destination, {4} -> dateofdeparture
req_link = '''http://developer.goibibo.com/api/search/?app_id={0}&app_key={1}&format=json&source={2}&destination={3}&dateofdeparture={4}&seatingclass=E&adults=1&children=0&infants=0&counter=0'''
req_link.format(apps_id,apps_key,source,destination,dod)

'http://developer.goibibo.com/api/search/?app_id=6eb4de47&app_key=32bb33028dcc07b634eaf02e0ce40360&format=json&source=JKT&destination=SIN&dateofdeparture=20190507&seatingclass=E&adults=1&children=0&infants=0&counter=0'

In [3]:
# connecting postgres offline
# postgres string ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}')
postgres_connection = ('postgresql://postgres:xxxxxxxxxxxxx')
# Create the connection
cnx = create_engine(postgres_connection)
df = pd.read_sql_query("""SELECT * from etl_config""", cnx)
df.head()

,id,source,destination,dod
0,1,SIN,JKT,20190507


Boleh pakai SQLachemy dan pandas, asal bisa ambil data dari database

In [4]:
conn = psycopg2.connect(postgres_connection)
cursor = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cursor.execute('''SELECT id, source, destination, dod FROM public.etl_config''')
rows = cursor.fetchall()
for row in rows:
    if row['id'] == 1:
        source = row['source']
        destination = row['destination']
        dod = row['dod']

In [5]:
# Create and Launch req_ready for getting url requests
req_ready = req_link.format(apps_id, apps_key, list(df['source'].values)[0],list(df['destination'].values)[0],list(df['dod'].values)[0])
result = requests.get(req_ready)


In [6]:
req_ready

'http://developer.goibibo.com/api/search/?app_id=6eb4de47&app_key=32bb33028dcc07b634eaf02e0ce40360&format=json&source=SIN&destination=JKT&dateofdeparture=20190507&seatingclass=E&adults=1&children=0&infants=0&counter=0'

In [7]:
 result.text

'{\n    "data": {\n        "returnflights": [], \n        "onwardflights": [\n            {\n                "origin": "SIN", \n                "rating": 0, \n                "DepartureTime": "", \n                "flightcode": "263", \n                "Group": "", \n                "farebasis": "E02H01", \n                "depterminal": "4", \n                "holdflag": "", \n                "CINFO": "air-SIN-JKT-20190507--1-0-0-E-0--", \n                "deptime": "10:20", \n                "codeshare": "", \n                "ibibopartner": "airasianew", \n                "duration": "1h 55m", \n                "platingcarrier": "", \n                "qtype": "", \n                "arrterminal": "2", \n                "flightno": "263", \n                "destination": "CGK", \n                "FlHash": "oQZ-263", \n                "stops": "0", \n                "seatsavailable": "49", \n                "carrierid": "QZ", \n                "airline": "Indonesia AirAsia", \n        

In [8]:
# replacing '\n' into an empty string ''
result_rep = result.text.replace('\n','')

# Decoding json with json.loads
result_json = json.loads(result.text)

In [9]:
# Coplying this result into JsonViewer
result_rep

'{    "data": {        "returnflights": [],         "onwardflights": [            {                "origin": "SIN",                 "rating": 0,                 "DepartureTime": "",                 "flightcode": "263",                 "Group": "",                 "farebasis": "E02H01",                 "depterminal": "4",                 "holdflag": "",                 "CINFO": "air-SIN-JKT-20190507--1-0-0-E-0--",                 "deptime": "10:20",                 "codeshare": "",                 "ibibopartner": "airasianew",                 "duration": "1h 55m",                 "platingcarrier": "",                 "qtype": "",                 "arrterminal": "2",                 "flightno": "263",                 "destination": "CGK",                 "FlHash": "oQZ-263",                 "stops": "0",                 "seatsavailable": "49",                 "carrierid": "QZ",                 "airline": "Indonesia AirAsia",                 "provider": "",                 "PromotionId": "

In [10]:
result_json['data']['onwardflights'][10]['fare']['totalfare']

5906

In [11]:
# Instantenate sum_fare and total_flight
sum_fare = 0
total_flight = 0

# Counting sum_fare and total_flight per day
for r in result_json['data']['onwardflights']:
    fare = r['fare']['totalfare']
    sum_fare += fare
    total_flight += 1

In [12]:
print("Total sum of fare cost is: ", sum_fare)
print("Total flight is: ", total_flight)
print("Average of sum fare cost per total flight is: ", sum_fare / total_flight)

Total sum of fare cost is:  5264027
Total flight is:  146
Average of sum fare cost per total flight is:  36054.979452054795


In [13]:
# Creating empty final_result
final_result = {}

# Creating int type on row 'dod'
dod = int(row['dod'])

# Filling final_result dict with function for
for i in range(0,8):
    str(dod)
    req_ready = req_link.format(apps_id, apps_key, source, destination, dod)
    result = requests.get(req_ready)
    result_json = json.loads(result.text)
    sum_fare = 0
    total_flight = 0
    for r in result_json['data']['onwardflights']:
        fare = r['fare']['totalfare']
        sum_fare += fare
        total_flight += 1
    average_fare = sum_fare / total_flight
    final_result[str(dod)] = average_fare
    dod +=1
    

In [14]:
final_result

{'20190507': 40367.62222222222,
 '20190508': 41170.21678321678,
 '20190509': 37130.6301369863,
 '20190510': 31390.030303030304,
 '20190511': 42182.90647482014,
 '20190512': 41145.792592592596,
 '20190513': 43479.04929577465,
 '20190514': 41689.74647887324}

In [15]:
df_average = pd.DataFrame.from_dict(final_result, orient='index', columns=['average_fare'])
df_average.reset_index(level=0, inplace=True)
df_average

,index,average_fare
0,20190507,40367.622222
1,20190508,41170.216783
2,20190509,37130.630137
3,20190510,31390.030303
4,20190511,42182.906475
5,20190512,41145.792593
6,20190513,43479.049296
7,20190514,41689.746479


In [20]:
df_avg = df_average.rename(columns={"index": "depdate"})
df_avg

,depdate,average_fare
0,20190507,40367.622222
1,20190508,41170.216783
2,20190509,37130.630137
3,20190510,31390.030303
4,20190511,42182.906475
5,20190512,41145.792593
6,20190513,43479.049296
7,20190514,41689.746479


In [22]:
df_avg.depdate = pd.to_datetime(df_avg.depdate)
df_avg.dtypes

depdate         datetime64[ns]
average_fare           float64
dtype: object

In [23]:
# with plotly

import plotly
import plotly.plotly as py
from plotly.grid_objs import Grid, Column
from plotly.tools import FigureFactory as ff
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

# Create a trace
plotly.tools.set_credentials_file(username='Rizdi21', api_key='QwNEljtKz2xcQkl1aY2I')


In [25]:
# Plotting average_fare

trace1 = go.Scatter(
x = df_avg['depdate'],
y = df_avg['average_fare'],
)

data = [trace1]

plotly.offline.iplot(data, filename='basic-line')

# Questions
1. extend with min price, max price, dan konversi ke rupiah

2. Buat line chart pergerakan point dari 4 klub di premier league di setiap *MatchDay* : Arsenal, Liverpool, Manchester City, Manchester United

In [ ]:
# Create and Launch req_ready for getting url requests
req_ready = req_link.format(apps_id, apps_key, list(df['source'].values)[0],list(df['destination'].values)[0],list(df['dod'].values)[0])
result = requests.get(req_ready)

# replacing '\n' into an empty string ''
result_rep = result.text.replace('\n','')

# Decoding json with json.loads
result_json = json.loads(result.text)


In [ ]:
# Accessing result_json directs to data/onwardflights
result_json

In [ ]:
# Get details on fare on flightrecord no. 0
result_json['data']['onwardflights'][0]['fare']

In [26]:
# Creating empty dataframe df_pesawat
df_pesawat_final = pd.DataFrame([])
date_of_departure = 20190507

for r in range(0,8):
    apps_id = '6eb4de47'
    apps_key = '32bb33028dcc07b634eaf02e0ce40360'
    source = 'SIN'
    destination = 'JKT'
    url_pesawat = 'https://developer.goibibo.com/api/search/?app_id=' + apps_id + '&app_key=' + apps_key + '&format=json&source=' + source + '&destination='+ destination + '&dateofdeparture=' + str(date_of_departure) + '&seatingclass=E&adults=1&children=0&infants=0&counter=0'  
    data_pesawat = requests.get(url_pesawat)
    date_of_departure += 1
# Filling empty entry df_pesawat by 
    for i in range(0,len(data_pesawat.json()['data']['onwardflights'])):
        x = [data_pesawat.json()['data']['onwardflights'][i]]
        y = [data_pesawat.json()['data']['onwardflights'][i]['fare']]
        df_pesawat1 = pd.DataFrame(x)[['origin','destination','depdate']]
        df_pesawat2 = pd.DataFrame(y)['totalfare']
        df_pesawat = pd.concat([df_pesawat1,df_pesawat2],axis='columns')
        df_pesawat_final = df_pesawat_final.append(df_pesawat)
    
df_pesawat_final['depdate'] = df_pesawat_final['depdate'].map(lambda z: str(z)[:-5])
df_pesawat_final = df_pesawat_final.reset_index(drop=True)
df_pesawat_final.head()

,origin,destination,depdate,totalfare
0,SIN,CGK,2019-05-07,3956
1,SIN,CGK,2019-05-07,3956
2,SIN,CGK,2019-05-07,3956
3,SIN,CGK,2019-05-07,3956
4,SIN,KUL,2019-05-07,5295


In [27]:
df_pesawat_final.tail()

,origin,destination,depdate,totalfare
1186,SIN,SGN,2019-05-14,28475
1187,SIN,HKG,2019-05-14,47762
1188,SIN,KUL,2019-05-14,13378
1189,SIN,KUL,2019-05-14,16689
1190,SIN,SUB,2019-05-14,19968


In [28]:
# Finding totalfare max in entire dataframe
df_pesawat_max = df_pesawat_final.groupby('depdate',as_index=False).max().rename(columns={'totalfare':'Lowest Fare (INR)'})
df_pesawat_min = df_pesawat_final.groupby('depdate',as_index=False).min().rename(columns={'totalfare':'Highest Fare (INR)'})
df_pesawat_max_min = pd.merge(df_pesawat_min,df_pesawat_max,on='depdate')
#df_pesawat = df_pesawat[['depdate','Highest Fare (INR)','Lowest Fare (INR)']]
df_pesawat_max_min

,depdate,origin_x,destination_x,Highest Fare (INR),origin_y,destination_y,Lowest Fare (INR)
0,2019-05-07,SIN,BKK,3956,SIN,XMN,394980
1,2019-05-08,SIN,ADD,3956,SIN,XMN,393994
2,2019-05-09,SIN,BKK,3956,SIN,XMN,396681
3,2019-05-10,SIN,ADD,3956,SIN,XMN,394980
4,2019-05-11,SIN,BKK,3956,SIN,XMN,397571
5,2019-05-12,SIN,BKK,3956,SIN,XMN,397571
6,2019-05-13,SIN,ADD,3956,SIN,XMN,334526
7,2019-05-14,SIN,BKK,3956,SIN,XMN,393994


In [29]:
df_pesawat = df_pesawat_max_min[['depdate','Highest Fare (INR)','Lowest Fare (INR)']]
df_pesawat

,depdate,Highest Fare (INR),Lowest Fare (INR)
0,2019-05-07,3956,394980
1,2019-05-08,3956,393994
2,2019-05-09,3956,396681
3,2019-05-10,3956,394980
4,2019-05-11,3956,397571
5,2019-05-12,3956,397571
6,2019-05-13,3956,334526
7,2019-05-14,3956,393994


In [35]:
df_pesawat.depdate = pd.to_datetime(df_pesawat.depdate)
df_pesawat.dtypes

C:\Users\Rizdi\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



depdate               datetime64[ns]
Highest Fare (INR)             int64
Lowest Fare (INR)              int64
dtype: object

In [37]:
df_pesawat.dtypes

depdate               datetime64[ns]
Highest Fare (INR)             int64
Lowest Fare (INR)              int64
dtype: object

In [38]:
df_pesawat_max_min_mean = pd.merge(df_pesawat,df_avg,on='depdate')

In [39]:
df_pesawat_max_min_mean

,depdate,Highest Fare (INR),Lowest Fare (INR),average_fare
0,2019-05-07,3956,394980,40367.622222
1,2019-05-08,3956,393994,41170.216783
2,2019-05-09,3956,396681,37130.630137
3,2019-05-10,3956,394980,31390.030303
4,2019-05-11,3956,397571,42182.906475
5,2019-05-12,3956,397571,41145.792593
6,2019-05-13,3956,334526,43479.049296
7,2019-05-14,3956,393994,41689.746479


In [41]:
df_pesawat_summary = df_pesawat_max_min_mean.rename(columns={"average_fare": "Average Fare (INR)"})
df_pesawat_summary

,depdate,Highest Fare (INR),Lowest Fare (INR),Average Fare (INR)
0,2019-05-07,3956,394980,40367.622222
1,2019-05-08,3956,393994,41170.216783
2,2019-05-09,3956,396681,37130.630137
3,2019-05-10,3956,394980,31390.030303
4,2019-05-11,3956,397571,42182.906475
5,2019-05-12,3956,397571,41145.792593
6,2019-05-13,3956,334526,43479.049296
7,2019-05-14,3956,393994,41689.746479


In [44]:
# Import requests, json
import requests
import json

# Calling URL
url = "https://api.exchangeratesapi.io/latest?symbols=INR,IDR"
response = requests.get(url)

# Reading Output
data = response.text

# Parse json - convert the string into JSON
parsed = json.loads(data)

# Extract data for IDR (Indonesian Rupiah) & INR (India Rupee)
idr_rate = parsed["rates"]["IDR"]
inr_rate = parsed["rates"]["INR"]

# IDR and INR ratio
Ratio = idr_rate / inr_rate

In [45]:
df_pesawat_summary_2 = df_pesawat_summary.rename(columns={"Highest Fare (INR)": "Lowest Fare (INR)", "Lowest Fare (INR)": "Highest Fare (INR)"})
df_pesawat_summary_2

,depdate,Lowest Fare (INR),Highest Fare (INR),Average Fare (INR)
0,2019-05-07,3956,394980,40367.622222
1,2019-05-08,3956,393994,41170.216783
2,2019-05-09,3956,396681,37130.630137
3,2019-05-10,3956,394980,31390.030303
4,2019-05-11,3956,397571,42182.906475
5,2019-05-12,3956,397571,41145.792593
6,2019-05-13,3956,334526,43479.049296
7,2019-05-14,3956,393994,41689.746479


In [63]:
df_pesawat_summary_2["Lowest Fare (IDR)"] = df_pesawat_summary_2["Lowest Fare (INR)"] * Ratio
df_pesawat_summary_2["Highest Fare (IDR)"] = df_pesawat_summary_2["Highest Fare (INR)"] * Ratio
df_pesawat_summary_2["Average Fare (IDR)"] = df_pesawat_summary_2["Average Fare (INR)"] * Ratio
df_pesawat_summary_3 = df_pesawat_summary_2.drop(['Lowest Fare (INR)', 'Highest Fare (INR)', 'Average Fare (INR)'], axis=1)
df_pesawat_summary_3

,depdate,Lowest Fare (IDR),Highest Fare (IDR),Average Fare (IDR)
0,2019-05-07,813991.884273,8.127162e+07,8.306096e+06
1,2019-05-08,813991.884273,8.106874e+07,8.471239e+06
2,2019-05-09,813991.884273,8.162162e+07,7.640048e+06
3,2019-05-10,813991.884273,8.127162e+07,6.458855e+06
4,2019-05-11,813991.884273,8.180474e+07,8.679612e+06
5,2019-05-12,813991.884273,8.180474e+07,8.466214e+06
6,2019-05-13,813991.884273,6.883252e+07,8.946308e+06
7,2019-05-14,813991.884273,8.106874e+07,8.578138e+06


In [64]:
# Plotting average_fare

trace4 = go.Scatter(
x = df_pesawat_summary_3['depdate'],
y = df_pesawat_summary_3['Lowest Fare (IDR)'],
)
trace5 = go.Scatter(
x = df_pesawat_summary_3['depdate'],
y = df_pesawat_summary_3['Highest Fare (IDR)'],
)
trace6 = go.Scatter(
x = df_pesawat_summary_3['depdate'],
y = df_pesawat_summary_3['Average Fare (IDR)'],
)

data = [trace4,trace5,trace6]

plotly.offline.iplot(data, filename='basic-line')